In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
from collections import Counter

In [3]:
import os

In [4]:
# 文件路径设置
base_dir = '../../../dataset/cnews/'
train_dir = os.path.join(base_dir,'cnews.train.txt')
test_dir = os.path.join(base_dir,'cnews.test.txt')
val_dir = os.path.join(base_dir,'cnews.val.txt')
vocab_dir = os.path.join(base_dir,'cnews.vocab.txt')

In [5]:
# 功能函数
# 打开文件
def open_file(filename,mode='r'):
    return open(filename,mode,encoding='utf-8')
# 读取文件
def read_file(filename):
    contents,labels = [],[]
    with open_file(filename) as f:
        for line in f:
            try:
                label,content = line.strip().split('\t')
                if content:
                    contents.append(list(content))
                    labels.append(label)
            except:
                pass
        return contents,labels

In [6]:
# 根据训练集构建词汇表，存储
def build_vocab(train_dir,vocab_dir,vocab_size=5000):
    data_train,_ = read_file(train_dir)
    all_data = []
    for content in data_train:
        all_data.extend(content)
    counter = Counter(all_data)
    count_pairs = counter.most_common(vocab_size - 1) # 此处减去1是因为PAD占据0位
    words,_ = list(zip(*count_pairs))
    # 添加一个<PAD>来将所有文本pad为同一长度
    words = ['<PAD>'] + list(words)
    open_file(vocab_dir,mode='w').write('\n'.join(words) + '\n')

In [7]:
# 读取词汇表
def read_vocab(vocab_dir):
    with open_file(vocab_dir) as f:
        words = [_.strip() for _ in f.readlines()]
    word_to_id = dict(zip(words,range(len(words))))
    return words,word_to_id

In [8]:
# 读取分类目录，目录固定
def read_category():
    categories = ['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
    cat_to_id = dict(zip(categories,range(len(categories))))
    return categories,cat_to_id

In [9]:
def to_words(content,words):
    '''
    将id表示的内容转换为文字
    '''
    return ''.join(words[x] for x in content)

In [24]:
def process_file(filename,word_to_id,cat_to_id,max_length=600):
    '''
    将文件转换为id表示
    '''
    contents,labels = read_file(filename)
    data_id,label_id = [],[]
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    print(type(label_id))
    print(len(label_id))
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = keras.preprocessing.sequence.pad_sequences(data_id,max_length)
    y_pad = keras.utils.to_categorical(label_id,num_classes=len(cat_to_id))
    
    return x_pad,y_pad

In [11]:
def batch_iter(x,y,batch_size=64):
    '''
    生成批次数据
    '''
    data_len = len(x)
    num_batch = int((data_len - 1)/batch_size) + 1
    
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x[indices]
    y_shuffle = y[indices]
    
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size,data_len)
        yield x_shuffle[start_id:end_id],y_shuffle[start_id:end_id]

In [12]:
# 数据探索 ，读取文本，查看文本信息
x_ori,y_ori = read_file(train_dir)
print(len(x_ori),len(y_ori))
x_val_ori,y_val_ori = read_file(val_dir)
print(len(x_val_ori),len(y_val_ori))

50000 50000
5000 5000


In [13]:
# 构建词汇表(数据集中已有)
# build_vocab(train_dir, vocab_dir, vocab_size=5000)

In [14]:
# 读取分类目录
categories,cat_to_id = read_category()
print(len(categories),len(cat_to_id))

10 10


In [15]:
# 输出类的个数
# 训练集
for i in categories:
    print(str(i) + ":" + str(y_ori.count(i)))
print("-"*50)
# 验证集
for i in categories:
    print(str(i) + ":" + str(y_val_ori.count(i)))

体育:5000
财经:5000
房产:5000
家居:5000
教育:5000
科技:5000
时尚:5000
时政:5000
游戏:5000
娱乐:5000
--------------------------------------------------
体育:500
财经:500
房产:500
家居:500
教育:500
科技:500
时尚:500
时政:500
游戏:500
娱乐:500


In [16]:
# 获取词汇表
words,word_to_id = read_vocab(vocab_dir) 

In [17]:
print(len(words),len(word_to_id)) # 由于words中有重复的元素导致长度不同

5000 4998


In [18]:
from collections import Counter

In [19]:
# temp = (str(k) + ":" + str(v) for k,v in dict(Counter(words).items()) if v > 1)

In [20]:
Counter(words).most_common(1)

[('', 3)]

In [25]:
# 建立训练集和测试集，将文本转换为id的形式，便于后续放入网络中
x_train,y_train = process_file(train_dir,word_to_id,cat_to_id,max_length=600)
x_val,y_val = process_file(val_dir,word_to_id,cat_to_id,max_length=600)
# print(x_train[0])
# print(y_train[0])
# print([len(x_train[i]) for i in range(3)])  # 查看单句长度

<class 'list'>
50000
<class 'list'>
5000


In [26]:
# 将id的形式转换为文本
print(to_words(x_train[0],words))

沈阳就有大雨，但幸好队伍上午的训练并没有受到任何干扰。下午6点，当球队抵达训练场时，大雨已经下了几个小时，而且丝毫没有停下来的意思。抱着试一试的态度，球队开始了当天下午的例行训练，25分钟过去了，天气没有任何转好的迹象，为了保护球员们，国奥队决定中止当天的训练，全队立即返回酒店。在雨中训练对足球队来说并不是什么稀罕事，但在奥运会即将开始之前，全队变得“娇贵”了。在沈阳最后一周的训练，国奥队首先要保证现有的球员不再出现意外的伤病情况以免影响正式比赛，因此这一阶段控制训练受伤、控制感冒等疾病的出现被队伍放在了相当重要的位置。而抵达沈阳之后，中后卫冯萧霆就一直没有训练，冯萧霆是7月27日在长春患上了感冒，因此也没有参加29日跟塞尔维亚的热身赛。队伍介绍说，冯萧霆并没有出现发烧症状，但为了安全起见，这两天还是让他静养休息，等感冒彻底好了之后再恢复训练。由于有了冯萧霆这个例子，因此国奥队对雨中训练就显得特别谨慎，主要是担心球员们受凉而引发感冒，造成非战斗减员。而女足队员马晓旭在热身赛中受伤导致无缘奥运的前科，也让在沈阳的国奥队现在格外警惕，“训练中不断嘱咐队员们要注意动作，我们可不能再出这样的事情了。”一位工作人员表示。从长春到沈阳，雨水一路伴随着国奥队，“也邪了，我们走到哪儿雨就下到哪儿，在长春几次训练都被大雨给搅和了，没想到来沈阳又碰到这种事情。”一位国奥球员也对雨水的“青睐”有些不解。
